# Kaggle MNIST digits

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

datadir = 'data/'

In [2]:
# Reminder Conv2d args: (in, out, kern, stride, padding)
model = nn.Sequential(
    nn.Conv2d(1, 32, 4, 2, 1, bias=False),
    nn.BatchNorm2d(32),
    nn.LeakyReLU(),
    nn.Conv2d(32, 64, 4, 2, 1, bias=False),
    nn.BatchNorm2d(64),
    nn.Flatten(),
    nn.Linear(64 * 7 * 7, 10),
    nn.ReLU(),
)

In [3]:
def train(model, loader, epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for imgs, labels in loader:
            optimizer.zero_grad()
            pred = model(imgs.reshape(-1, 1, 28, 28))
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

def evaluate(model, loader):
    with torch.no_grad():
        total = 0
        for imgs, labels in loader:
            pred = model(imgs.reshape(-1, 1, 28, 28))
            total += (pred.argmax(dim=1) == labels).sum()
        print(total.item() / 7000)

In [4]:
train_dataset = pd.read_csv(f'{datadir}/train.csv')
labels = torch.tensor(train_dataset.pop('label').to_numpy(), dtype=torch.int64)
imgs = torch.tensor(train_dataset.to_numpy() / 255, dtype=torch.float32)
train_loader = DataLoader(list(zip(imgs[:35000], labels[:35000])), batch_size=10, shuffle=True)
eval_loader = DataLoader(list(zip(imgs[35000:], labels[35000:])), batch_size=10, shuffle=False)

In [5]:
evaluate(model, eval_loader)
train(model, train_loader, 1, 0.0001)
evaluate(model, eval_loader)
train(model, train_loader, 5, 0.0001)
evaluate(model, eval_loader)

0.08271428571428571
0.9624285714285714
0.9715714285714285
